<a href="https://colab.research.google.com/github/bessabra/PRO2PRO/blob/main/professores_pioneiros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fuzzywuzzy
!pip install levenshtein
!pip install xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.0/174.0 KB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 44.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 KB 4.4 MB/s eta 0:00:00


`ListaSeiRevisao.ipynb`


---



---



In [2]:
import pandas      as pd
from   fuzzywuzzy  import fuzz
from   datetime    import datetime, timedelta
import xlsxwriter
import chardet
import pyarrow.parquet as pq


In [3]:
parquet_file = r'/content/Prof Pioneiros Referencia tudo.parquet'

base_ref_parquet = pd.read_parquet(parquet_file)

In [4]:
# Adiciona uma cópia da coluna SIAPE
base_ref_parquet.insert(11, 'IT-NO-SIAPE', base_ref_parquet['Siape'])

# Altera o nome da coluna "Siape" para "SIAPE", dessa forma o nome ficará igual entre a basedf e baserq
base_ref_parquet = base_ref_parquet.rename(
    columns = {'Siape':'SIAPE'}
    )

# Concatena Cargo e Cargo.1 em IT-CO-CARGO-EMPREGO
base_ref_parquet['IT-CO-CARGO-EMPREGO'] = base_ref_parquet['Cargo'].astype(str) + ' - ' + base_ref_parquet['Cargo.1'].astype(str)

Rodar a partir daqui após carregar Prof Pioneiros Referencia tudo.xlsx


---



In [5]:
values_dict = {
    '40801_26_12': 'AP',
    '40801_09_01': 'AP',
    '40801_23_01': 'AP',
    '40803_27_12': 'RO',
    '40804_27_12': 'RR',
    '40804_09_01': 'RR'
    }

origem =   {
    '40801_26_12': 'LISTA_SEI_PROFESSORES_40801_26_12_2022',
    '40801_09_01': 'LISTA_SEI_PROFESSORES_COMPLEMENTO_40801_09_01_2023',
    '40801_23_01': 'LISTA_SEI_PROFESSORES_COMPLEMENTO_40801_23_01_2023',
    '40803_27_12': 'LISTA_SEI_PROFESSORES_40803_27_12_2022',
    '40804_27_12': 'LISTA_SEI_PROFESSORES_40804_27_12_2022',
    '40804_09_01': 'LISTA_SEI_PROFESSORES_40804_09_01_2023'
    }

sheetlist = [
    '40801_26_12',             
    '40801_09_01',             
    '40801_23_01',            
    '40803_27_12',             
    '40804_27_12',             
    '40804_09_01'
    ]
    
base_sheet_list = []

for sheet in sheetlist:
    baseListaSEI = pd.read_excel('/content/LISTAS_SEI_GERAL.xlsx', sheet_name=sheet, index_col=None)
    if sheet in values_dict and sheet in origem:
        baseListaSEI['ESTADO'] = values_dict[sheet]
        baseListaSEI['LISTA'] = origem[sheet]
    base_sheet_list.append(baseListaSEI)

baserq = pd.concat(base_sheet_list, axis=0, ignore_index=True)

Original = baserq.copy()
Original['DATA DO REQUERIMENTO'] = pd.to_datetime(Original['DATA DO REQUERIMENTO'], errors="coerce")

In [6]:
#Lista SEI de requerimentos para selecionar 40801-AP 40803-RO 40804-RR
baserq['DATA DO REQUERIMENTO'] = pd.to_datetime(baserq['DATA DO REQUERIMENTO'], errors="coerce")

In [7]:
# Renomeia as colunas para evitar conflitos no merge
baserq = baserq.rename(
    columns = {'DESCRIÇÃO':'ASSUNTO'}
    )

In [8]:
# Muda os caracteres para maiúsculos
baserq['INTERESSADO']           = baserq['INTERESSADO'].str.upper()
baserq['DATA DO REQUERIMENTO']  = baserq['DATA DO REQUERIMENTO'].dt.strftime('%d/%m/%Y')

In [9]:
# Junta as planilhas basedf e baserq pra identificar correspondência de SIAPE
mixResultadoGeral = baserq.merge(
    base_ref_parquet, on=['SIAPE'], 
    how='outer', 
    # how='right', 
    suffixes=['', '_'], 
    indicator=True
    )

# Adiciona uma coluna com legendas para cada situação
mixResultadoGeral.loc[(mixResultadoGeral['_merge'] == 'left_only'), 'CONDIÇÃO']   = 'Número SIAPE não encontrado em Prof Pioneiros Referencia tudo'
mixResultadoGeral.loc[(mixResultadoGeral['_merge'] == 'both'), 'CONDIÇÃO']        = 'Número SIAPE encontrado em Prof Pioneiros Referencia tudo'


# Lista de colunas para manter
colunas_manter = [
    'PROCESSO SEI', 
    'DATA DO REQUERIMENTO', 
    'SIAPE', 
    'INTERESSADO',
    'ASSUNTO',
    'ESTADO',
    'LISTA',
    'GR-MATRICULA',
    'Órgão',
    'IT-NU-IDEN-SERV-ORIGEM',
    'IT-NO-SIAPE',
    'IT-NO-SERVIDOR',
    'IT-NU-CPF',
    '_merge',
    'IT-CO-CARGO-EMPREGO'
    ]

# Exclusão das colunas que não estão na lista
mixResultadoGeral = mixResultadoGeral[colunas_manter]


# Calcula a similaridade entre os nomes dos interessados e dos servidores usando cálculo de distância de Levenshtein. Indentificar se o servidor foi o requerente.
mixResultadoGeral['SIMILARIDADE'] = mixResultadoGeral.apply(lambda row: fuzz.partial_ratio(str(row['INTERESSADO']),str(row['IT-NO-SERVIDOR']))if(isinstance(row['INTERESSADO'],str) and isinstance(row['IT-NO-SERVIDOR'],str))else None, axis=1)


In [10]:
colunas_manter_sim = [
    'PROCESSO SEI', 
    'DATA DO REQUERIMENTO', 
    'SIAPE', 
    'INTERESSADO',
    'ASSUNTO',
    'ESTADO',
    'LISTA',
    'GR-MATRICULA',
    'Órgão',
    'IT-NU-IDEN-SERV-ORIGEM',
    'IT-NO-SIAPE',
    'IT-NO-SERVIDOR',
    'IT-NU-CPF', 
    '_merge',
    'IT-CO-CARGO-EMPREGO',
    'SIMILARIDADE'
    ]

SimilaridadeMenor50   = mixResultadoGeral.loc[(mixResultadoGeral['SIMILARIDADE'] <= 50)]
SimilaridadeMaior50   = mixResultadoGeral.loc[(mixResultadoGeral['SIMILARIDADE'] > 50)]
SimilaridadeMenor50   = SimilaridadeMenor50[colunas_manter_sim]
SimilaridadeMaior50   = SimilaridadeMaior50[colunas_manter_sim]

In [11]:
# Cria uma nova planilha somente com os servidores não encontrados na Prof Pioneiros Referencia tudo
servNao = mixResultadoGeral.loc[(mixResultadoGeral['_merge'] == 'left_only')]
# servNao = SimilaridadeMenor50.loc[(SimilaridadeMenor50['_merge'] == 'both')]

# Cria uma nova planilha somente com os servidores encontrados Prof Pioneiros Referencia tudo
# servEnc = mixResultadoGeral.loc[(mixResultadoGeral['_merge'] == 'both')]
servEncMaior = SimilaridadeMaior50.loc[(SimilaridadeMaior50['_merge'] == 'both')]
servEncMenor = SimilaridadeMenor50.loc[(SimilaridadeMenor50['_merge'] == 'both')]
# Gerando as planilhas de Encontrados e Não Encontrados
now = datetime.now()

if not servNao.empty:
    servNaoEnc = f"servNao{now.strftime('%Y-%m-%d %H:%M:%S')}.xlsx"
    servNao.to_excel(servNaoEnc, index=False)
else:
    print("O DataFrame servNao está vazio, arquivo não será criado.")

O DataFrame servNao está vazio, arquivo não será criado.


In [12]:
# Exclue a coluna _merge
servEncMaior.drop('_merge', axis = 1, inplace=True)
servEncMenor.drop('_merge', axis = 1, inplace=True)

In [13]:
# Agrupa as primeiras ocorrências dos SIAPEs alvos
resultadoMaior = servEncMaior.groupby('SIAPE').first().reset_index()
resultadoMenor = servEncMenor.groupby('SIAPE').first().reset_index()

Original['DATA DO REQUERIMENTO']        = Original['DATA DO REQUERIMENTO'].dt.strftime('%d/%m/%Y')

# Colunas
cols      = resultadoMaior.columns.tolist()
colsmenor = resultadoMenor.columns.tolist()

# Colocando SIAPE na posição 2
cols.insert(2, cols.pop(cols.index('SIAPE')))
colsmenor.insert(2, colsmenor.pop(colsmenor.index('SIAPE')))

# Atribuindo as novas colunas
resultadoMaior = resultadoMaior[cols]
resultadoMenor = resultadoMenor[colsmenor]

In [14]:
#Cria uma base contendo apenas as duplicadas
resultadoDuplicadas = Original[Original.duplicated(subset=['SIAPE'], keep=False)]

In [15]:
# Criar um arquivo Excel com ExcelWriter, usando a engine='xlsxwriter'
now = datetime.now()
Resultado = f"01 Lista SEI {now.strftime('%Y-%m-%d %H:%M:%S')}.xlsx"
writer = pd.ExcelWriter(Resultado, engine='xlsxwriter')

# Escreva os dataframes no arquivo Excel
Original.to_excel(writer, sheet_name='ORIGINAL', index=False)
resultadoMaior.to_excel(writer, sheet_name='TRATADA', index=False)
resultadoDuplicadas.to_excel(writer, sheet_name='DUPLICADAS', index = False)
resultadoMenor.to_excel(writer, sheet_name='NOMES_DIFERENTES', index=False)

# Definir formato, filtros e cabeçalho para cada planilha
for sheet_name in ['ORIGINAL', 'TRATADA', 'NOMES_DIFERENTES', 'DUPLICADAS']:
    worksheet = writer.sheets[sheet_name]

    # Definir o formato de célula centralizado
    cell_format = writer.book.add_format({'align': 'center', 'valign': 'center'})

    # Aplicar o formato de célula centralizado às colunas da planilha
    num_cols = len(Original.columns)
    num_rows = len(Original)

     # Ajustar a largura da coluna de modo a mostrar todo o texto
    for col_num, col_width in enumerate(Original.iloc[:, col_num].astype(str).str.len().max() for col_num in range(len(Original.columns))):
        worksheet.set_column(col_num, col_num, col_width + 1, cell_format)

    for row_num in range(num_rows):
        worksheet.set_row(row_num, None, cell_format)

    # Adicionar filtros às colunas da planilha
    worksheet.autofilter(0, 0, num_rows - 1, num_cols - 1)

    # Congelar a primeira linha
    worksheet.freeze_panes(1, 0)

    # Definir a cor azul para os nomes das colunas
    header_format = writer.book.add_format({'bold': True, 'bg_color': '#0070C0', 'color': 'white'})
    for col_num, value in enumerate(Original.columns.values):
        worksheet.write(0, col_num, value, header_format)

# Salve o arquivo
writer.save()

Aqui é o trecho que filtra a carreira  `consolidaLista.ipynb`


---




---



In [16]:
# Lista de colunas para manter
colunas_manter = [
    'PROCESSO SEI',
    'DATA DO REQUERIMENTO', 
    'SIAPE', 
    'INTERESSADO',
    'IT-NO-SERVIDOR',
    'IT-NU-CPF',
    'IT-CO-CARGO-EMPREGO',
    'LISTA', 
    'ESTADO'
    ]

# Exclusão das colunas que não estão na lista
base_carreira = resultadoMaior[colunas_manter].copy()

In [17]:
base_carreira = base_carreira.rename(columns = {'IT-NO-SERVIDOR':'SERVIDOR', 'IT-NU-CPF':'CPF', 'IT-CO-CARGO-EMPREGO':'CARGO'})


In [18]:
# Identifica os seis valores possíveis de cargo e carreira
CARGO_LISTA = ['709001 - CARREIRA MAGIST ENS BASICO EX-TERRITORIO - PROFESSOR DO ENSINO BASICO EX-TERRITORIO',
               '709001 - PROFESSOR DO ENSINO BASICO EX-TERRITORIO',
               '707001 - PROFESSOR ENS BASICO TECN TECNOLOGICO',
               '707001 - CARREIRA MAGIST ENS BAS TEC TECNOL - PROFESSOR ENS BASICO TECN TECNOLOGICO',
               '704001 - CAR.MAG.ENSINO BASICO EX-TERRITORIO - PROFESSOR DO ENSINO BASICO EX-TERRITORIO',
               '704001 - PROFESSOR DO ENSINO BASICO EX-TERRITORIO']

for cargo in CARGO_LISTA:
    base_carreira.loc[~base_carreira['CARGO'].isin(CARGO_LISTA), 'CARREIRA'] = 'FORA DO ESCOPO'

In [19]:
# Elimina carreiras fora do escopo
foraEscopo = base_carreira.loc[(base_carreira['CARREIRA'] == 'FORA DO ESCOPO')].copy()

# Cria uma base eliminando carreiras fora do escopo
base_carreira = base_carreira[base_carreira.CARREIRA != 'FORA DO ESCOPO']

In [20]:
base_consolidada = base_carreira.drop('CARREIRA', axis=1).copy()

In [21]:
# Gerando a planilha ResultadoRevisao
now = datetime.now()

# Usando o ExcelWriter, cria um doc .xlsx, usando engine='xlsxwriter'
now = datetime.now()
Resultado = f"02 LISTA_SEI_PROFESSORES_CONSOLIDADA{now.strftime('%Y-%m-%d %H:%M:%S')}.xlsx"
writer = pd.ExcelWriter(Resultado, engine='xlsxwriter')

# Armazena cada df em uma planilha diferente do mesmo arquivo
base_consolidada.to_excel(writer, sheet_name='CONSOLIDADA', index = False)
foraEscopo.to_excel(writer, sheet_name='CARREIRA_FORA_DO_ESCOPO', index = False)


# Definir formato, filtros e cabeçalho para cada planilha
for sheet_name in ['CONSOLIDADA','CARREIRA_FORA_DO_ESCOPO']:
    worksheet = writer.sheets[sheet_name]
  
    # Definir o formato de célula centralizado
    cell_format   = (
        writer
        .book
        .add_format(
            {'align': 'center',
             'valign': 'center'}
             )
        )
    
    header_format = (
        writer
        .book
        .add_format(
            {'bold': True,
             'bg_color': '#0070C0',
             'color': 'white',
             'align': 'center',
             'valign': 'center'}
             )
        )
    
    worksheet.set_column(0, 0, options={'hidden': True})

    # Aplicar o formato de célula centralizado às colunas da planilha
    num_cols = len(base_consolidada.columns)
    num_rows = len(base_consolidada)

    # Ajustar a largura da coluna de modo a mostrar todo o texto
    for col_num in range(len(base_consolidada.columns)):
        col_width = (
            base_consolidada
            .iloc[:, col_num]
            .astype(str)
            .str
            .len()
            .max()
            )
        
        worksheet.set_column(col_num, col_num, col_width, cell_format)

    for row_num in range(num_rows):
        worksheet.set_row(row_num, None, cell_format)

    # Adicionar filtros às colunas da planilha
    worksheet.autofilter(0, 0, num_rows - 1, num_cols - 1)

    # Congelar a primeira linha
    worksheet.freeze_panes(1, 0)
    
    # # Definir a cor azul para os nomes das colunas
    for col_num, value in enumerate(base_consolidada.columns):
        worksheet.write(0, col_num, value, header_format)


# Salve o arquivo
writer.save()

Aqui é feita a conferência com Dados Funcionais.xlsx `DadosMixSei.ipynb`

---



---



In [22]:
# Cria a base_funcionais com os dados de Dados Funcionais v15.02.xlsx
base_funcionais = pd.read_excel('/content/Dados Funcionais v15.02.xlsx')

In [23]:
# Altera o nome da coluna "Siape" para "SIAPE", dessa forma o nome ficará igual entre a base_funcionais e base_consolidada
base_funcionais = base_funcionais.rename(
    columns = {'Siape':'SIAPE'}
    )

In [24]:
base_funcionais['IT-DA-OCOR-INATIVIDADE-SERV'] = pd.to_datetime(base_funcionais['IT-DA-OCOR-INATIVIDADE-SERV'], errors="coerce")
base_funcionais['IT-DA-OBITO']       = pd.to_datetime(base_funcionais['IT-DA-OBITO'], errors="coerce")

In [25]:
# Junta as planilhas basedf e baserq pra identificar correspondência de SIAPE
consolidadeMixFuncionais = base_consolidada.merge(
    base_funcionais, on=['SIAPE'], 
    how='outer', 
    suffixes=['_dados_funcionais', '_lista'], 
    indicator=True
    )

In [49]:
# Renomeia a coluna _merge
consolidadeMixFuncionais = consolidadeMixFuncionais.rename(
    columns = {'_merge':'CONDIÇÃO'}
    )

# Renomeia as categorias da coluna condição
consolidadeMixFuncionais['CONDIÇÃO'] = (
   consolidadeMixFuncionais['CONDIÇÃO']
   .cat
   .rename_categories(
       { 'right_only' :'Servidor_não_fez_requerimento',
        'left_only'   :'Servidor_não_encontrado_em_Dados_Funcionais',
        'both'        :'Servidor_encontrado_em_Dados_Funcionais'
        })
   ).copy()

# Cria duas novas bases de servidor_não_enconcontrado e servidor_enconcontrado
servidor_não_enconcontrado  =(
    consolidadeMixFuncionais
    .query('CONDIÇÃO == "Servidor_não_encontrado_em_Dados_Funcionais"')
    .copy()
    )
servidor_enconcontrado      = (
    consolidadeMixFuncionais
    .query('CONDIÇÃO == "Servidor_encontrado_em_Dados_Funcionais"')
    .copy()
    )

In [28]:
# Gerando a planilha ResultadoRevisao
now = datetime.now()

# Usando o ExcelWriter, cria um doc .xlsx, usando engine='xlsxwriter'
ResultadoRevisao = f"03 Merge Lista e Dados - Resultado {now.strftime('%Y-%m-%d %H:%M:%S')}.xlsx"
writer = pd.ExcelWriter(ResultadoRevisao, engine='xlsxwriter')

# Armazena cada df em uma planilha diferente do mesmo arquivo
servidor_enconcontrado.to_excel(writer, sheet_name='ENCONTRADOS_DADOS_FUN', index= False)
servidor_não_enconcontrado.to_excel(writer, sheet_name='NÃO_ENCONTRADOS_DADOS_FUN', index= False)


# Definir formato, filtros e cabeçalho para cada planilha
for sheet_name in ['ENCONTRADOS_DADOS_FUN', 'NÃO_ENCONTRADOS_DADOS_FUN']:
    worksheet = writer.sheets[sheet_name]
  
    # Definir o formato de célula centralizado
    cell_format   = (
        writer
        .book
        .add_format(
            {'align': 'center',
             'valign': 'center'}
             )
        )
    
    header_format = (
        writer
        .book
        .add_format(
            {'bold': True,
             'bg_color': '#0070C0',
             'color': 'white',
             'align': 'center',
             'valign': 'center'}
             )
        )
    
    worksheet.set_column(0, 0, options={'hidden': True})

    # Aplicar o formato de célula centralizado às colunas da planilha
    num_cols = len(servidor_enconcontrado.columns)
    num_rows = len(servidor_enconcontrado)

    # Ajustar a largura da coluna de modo a mostrar todo o texto
    for col_num in range(len(servidor_enconcontrado.columns)):
        col_width = (
            servidor_enconcontrado
            .iloc[:, col_num]
            .astype(str)
            .str
            .len()
            .max()
            )
        
        worksheet.set_column(col_num, col_num, col_width, cell_format)

    for row_num in range(num_rows):
        worksheet.set_row(row_num, None, cell_format)

    # Adicionar filtros às colunas da planilha
    worksheet.autofilter(0, 0, num_rows - 1, num_cols - 1)

    # Congelar a primeira linha
    worksheet.freeze_panes(1, 0)
    
    # # Definir a cor azul para os nomes das colunas
    for col_num, value in enumerate(servidor_enconcontrado.columns):
        worksheet.write(0, col_num, value, header_format)


# Salve o arquivo
writer.save()



---



---



---



In [50]:
base_servidor_enconcontrado = servidor_enconcontrado.copy()

In [52]:
# Renomeia as colunas sem underline
base_servidor_enconcontrado = base_servidor_enconcontrado.rename(
    columns = {
        'DATA DO REQUERIMENTO':'DATA_DO_REQUERIMENTO',
        'PROCESSO SEI':'PROCESSO_SEI',
        'IT-DA-OCOR-EXCLUSAO-SERV':'DATA_EXCLUSÃO',
        'Cad Ativo/Excluído':'SITUAÇÃO_CADASTRO',
        'IT-DA-OBITO':'OBITO',
        'IT-DA-OCOR-INATIVIDADE-SERV':'INATIVIDADE',
        'IT-CO-SITUACAO-SERVIDOR':'SITUAÇÃO',
        'IT-CO-OCOR-EXCLUSAO':'EXCLUSÃO'
        }
    )

In [54]:
#Mudando o tipo de veriavel para datetime
base_servidor_enconcontrado['INATIVIDADE']           = pd.to_datetime(base_servidor_enconcontrado['INATIVIDADE'], errors="coerce")
base_servidor_enconcontrado['OBITO']                 = pd.to_datetime(base_servidor_enconcontrado['OBITO'], errors="coerce")
base_servidor_enconcontrado['DATA_DO_REQUERIMENTO']  = pd.to_datetime(base_servidor_enconcontrado['DATA_DO_REQUERIMENTO'], errors="coerce")

#Subtrai 5 anos da Data Requerimento
base_servidor_enconcontrado['DR5'] = base_servidor_enconcontrado['DATA_DO_REQUERIMENTO'] - timedelta(days=1825)

In [55]:
# Requerimento antes da aposentadoria ou obito
base_servidor_enconcontrado.loc[(base_servidor_enconcontrado['DATA_DO_REQUERIMENTO'] < base_servidor_enconcontrado['INATIVIDADE']) | (base_servidor_enconcontrado['DATA_DO_REQUERIMENTO'] < base_servidor_enconcontrado['OBITO']), 'DRstatus'] ='01 - ATIVO PERMANENTE ou 08 - ATIVO EM OUTRO ORGAO'
base_servidor_enconcontrado.loc[(base_servidor_enconcontrado['DATA_DO_REQUERIMENTO'] > base_servidor_enconcontrado['INATIVIDADE']) | (base_servidor_enconcontrado['DATA_DO_REQUERIMENTO'] > base_servidor_enconcontrado['OBITO']), 'DRstatus'] ='02 – APOSENTADO ou 15 - INSTITUIDOR PENSAO'
base_servidor_enconcontrado.loc[(base_servidor_enconcontrado['DR5'] < base_servidor_enconcontrado['INATIVIDADE']) | (base_servidor_enconcontrado['DR5'] < base_servidor_enconcontrado['OBITO']), 'DR5status'] ='01 - ATIVO PERMANENTE ou 08 - ATIVO EM OUTRO ORGAO'
base_servidor_enconcontrado.loc[(base_servidor_enconcontrado['DR5'] > base_servidor_enconcontrado['INATIVIDADE']) | (base_servidor_enconcontrado['DR5'] > base_servidor_enconcontrado['OBITO']), 'DR5status'] ='02 – APOSENTADO ou 15 - INSTITUIDOR PENSAO'

# Situação após o filtro de intervalo de cobertura
base_servidor_enconcontrado.loc[(base_servidor_enconcontrado['SITUAÇÃO'] == '01 - ATIVO PERMANENTE'), 'RESULTADO'] ='ANÁLISE'
base_servidor_enconcontrado.loc[(base_servidor_enconcontrado['DRstatus'] == '02 – APOSENTADO ou 15 - INSTITUIDOR PENSAO') & (base_servidor_enconcontrado['DR5status']         == '02 – APOSENTADO ou 15 - INSTITUIDOR PENSAO'), 'RESULTADO'] ='INDEFERIDO'
base_servidor_enconcontrado.loc[(base_servidor_enconcontrado['DRstatus'] == '02 – APOSENTADO ou 15 - INSTITUIDOR PENSAO') & (base_servidor_enconcontrado['DR5status']         == '01 - ATIVO PERMANENTE ou 08 - ATIVO EM OUTRO ORGAO'), 'RESULTADO'] ='A DECIDIR'
base_servidor_enconcontrado.loc[(base_servidor_enconcontrado['DRstatus'] == '01 - ATIVO PERMANENTE ou 08 - ATIVO EM OUTRO ORGAO') & (base_servidor_enconcontrado['DR5status']  == '01 - ATIVO PERMANENTE ou 08 - ATIVO EM OUTRO ORGAO'), 'RESULTADO'] ='ANÁLISE'

In [56]:
colunas_conv_data = ['DATA_DO_REQUERIMENTO', 
                     'OBITO',
                     'INATIVIDADE',
                     'DATA_EXCLUSÃO',
                     'DR5']       

#Substituindo
for coluna in colunas_conv_data:
  base_servidor_enconcontrado[coluna] = pd.to_datetime(base_servidor_enconcontrado[coluna]).dt.strftime('%d/%m/%Y')

In [57]:
colunas_manter = ['RESULTADO',
                  'PROCESSO_SEI',
                  'ESTADO', 
                  'DATA_DO_REQUERIMENTO', 
                  'SIAPE', 
                  'INTERESSADO',
                  'CPF',
                  'SERVIDOR',
                  'CARGO',
                  'LISTA',
                  'SITUAÇÃO',
                  'OBITO', 
                  'INATIVIDADE',
                  'EXCLUSÃO',
                  'DATA_EXCLUSÃO',
                  'SITUAÇÃO_CADASTRO',
                  'DR5',
                  'DRstatus',
                  'DR5status']              											 				

# Exclusão das colunas que não estão na lista
base = base_servidor_enconcontrado[colunas_manter].copy()

In [58]:
# base_resumo = pd.concat([baseC, baseE, baseI, baseD], ignore_index=True)
resultado = base['SITUAÇÃO_CADASTRO'].value_counts().reset_index()
resultado.columns = ["SITUAÇÃO_CADASTRO_GERAL", "Quantidade"]


In [59]:
resultado_cad_ativo = (
    base
    .query('SITUAÇÃO_CADASTRO == "Cad Ativo" | SITUAÇÃO_CADASTRO == "Cad Ativo Inst"')
    .copy()
    )

resultado_cad_excluido = (
    base
    .query('SITUAÇÃO_CADASTRO == "Cad Excluído" | SITUAÇÃO_CADASTRO == "Cad Excluído Inst"')
    .copy()
    )


In [60]:
resultado_analise = baseAnalise  =(
    base
    .query('RESULTADO == "ANÁLISE"')
    .copy()
    )
resultado_indeferido = baseIdeferido  =(
    base
    .query('RESULTADO == "INDEFERIDO"')
    .copy()
    )

resultado_decidrir = baseADecidir  =(
    base
    .query('RESULTADO == "A DECIDIR"')
    .copy()
    )

resultado_faltou = baseFaltou  =(
    base
    .query('RESULTADO != "ANÁLISE" & RESULTADO != "INDEFERIDO" & RESULTADO != "A DECIDIR"')
    .copy()
    )


In [61]:
resultado2 = baseAnalise['SITUAÇÃO_CADASTRO'].value_counts().reset_index()
resultado2.columns = ["Análise", "Quantidade"]

resultado3 = baseIdeferido['SITUAÇÃO_CADASTRO'].value_counts().reset_index()
resultado3.columns = ["Ideferidos", "Quantidade"]

resultado4 = baseADecidir['SITUAÇÃO_CADASTRO'].value_counts().reset_index()
resultado4.columns = ["A Decidir", "Quantidade"]

resultado5 = resultado_faltou['ESTADO'].value_counts().reset_index()
resultado5.columns = ["Faltaram", "Quantidade"]

resultados = [resultado, resultado2, resultado3, resultado4, resultado5]

Resumo_de_resultados = pd.concat(resultados, axis=1)

In [66]:
# # Criar um arquivo Excel com ExcelWriter, usando a engine='xlsxwriter'
now = datetime.now()
ResultadoPreliminar = f"04 Análise Preliminar - Resultado {now.strftime('%Y-%m-%d %H:%M:%S')}.xlsx"
writer = pd.ExcelWriter(ResultadoPreliminar, engine='xlsxwriter')

# Escreva os dataframes no arquivo Excel
baseAnalise.to_excel            (writer, sheet_name ='ANÁLISE_NIVEL_4'        , index=False)
baseIdeferido.to_excel          (writer, sheet_name ='INDEFERIDO_NIVEL_4'     , index=False)
baseADecidir.to_excel           (writer, sheet_name ='DECIDIR_NIVEL_4'        , index=False)
resultado_cad_ativo.to_excel    (writer, sheet_name ='CAD_ATIVO_NIVEL_3'      , index=False)
resultado_cad_excluido.to_excel (writer, sheet_name ='CAD_EXCLUÍDO_NIVEL_3'   , index=False)
resultado_faltou.to_excel       (writer, sheet_name ='NÃO_ENCONTRADOS', index=False)

Resumo_de_resultados.to_excel(writer, sheet_name ='RESUMO', index=False)

cell_format   = writer.book.add_format({'align': 'center', 'valign': 'center'})
for row_num in range(len(Resumo_de_resultados)):
    writer.sheets['RESUMO'].set_row(row_num, None, cell_format)

for col_num in range(len(Resumo_de_resultados.columns)):
    col_width = Resumo_de_resultados.iloc[:, col_num].astype(str).str.len().max()
    writer.sheets['RESUMO'].set_column(col_num, col_num, col_width, cell_format)

# Definir formato, filtros e cabeçalho para cada planilha
for sheet_name in ['ANÁLISE_NIVEL_4', 'INDEFERIDO_NIVEL_4', 'DECIDIR_NIVEL_4', 'NÃO_ENCONTRADOS', 'CAD_ATIVO_NIVEL_3', 'CAD_EXCLUÍDO_NIVEL_3']:
    worksheet = writer.sheets[sheet_name]
  
    # Definir o formato de célula centralizado
    header_format = writer.book.add_format({'bold': True, 'bg_color': '#0070C0', 'color': 'white', 'align': 'center', 'valign': 'center'})
    worksheet.set_column(0, 0, options={'hidden': True})

    # Aplicar o formato de célula centralizado às colunas da planilha
    num_cols = len(baseAnalise.columns)
    num_rows = len(baseAnalise)

    # Ajustar a largura da coluna de modo a mostrar todo o texto
    for col_num in range(len(baseAnalise.columns)):
        col_width = baseAnalise.iloc[:, col_num].astype(str).str.len().max()
        worksheet.set_column(col_num, col_num, col_width, cell_format)

    for row_num in range(num_rows):
        worksheet.set_row(row_num, None, cell_format)

    # Adicionar filtros às colunas da planilha
    worksheet.autofilter(0, 0, num_rows - 1, num_cols - 1)

    # Congelar a primeira linha
    worksheet.freeze_panes(1, 0)
    
    # # Definir a cor azul para os nomes das colunas
    for col_num, value in enumerate(baseAnalise.columns):
        worksheet.write(0, col_num, value, header_format)


# Salve o arquivo
writer.save()



---



---



---



In [70]:
base  = pd.read_excel('/content/CACOPOSPRO V15.02.xlsx')
base_analise = baseAnalise.copy()


In [71]:
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72908 entries, 0 to 72907
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Órgão             72908 non-null  int64 
 1   SIAPE             72908 non-null  int64 
 2   CPF               72908 non-null  int64 
 3   SiapeCad          72908 non-null  int64 
 4   Nome              72908 non-null  object
 5   Cargo             72908 non-null  object
 6   Cargo Inicial     71755 non-null  object
 7   Cargo Final       63151 non-null  object
 8   Posicionamento    49742 non-null  object
 9   Vigencia Inicial  49742 non-null  object
 10  Vigencia Final    48295 non-null  object
 11  Vigencia Status   49742 non-null  object
dtypes: int64(4), object(8)
memory usage: 6.7+ MB


In [72]:
# Renomeia as colunas sem underline
base = base.rename(
    columns = {
        'Vigencia Final':'VIGENCIA_FINAL',
        'Vigencia Inicial':'VIGENCIA_INICIAL'
        }
    )

Redefinição de tipos das variáveis.

In [73]:
colunas_conv_data_base = ['VIGENCIA_INICIAL', 
                          'VIGENCIA_FINAL']

colunas_conv_data_base2 = ['DATA_DO_REQUERIMENTO',
                          'OBITO',
                          'INATIVIDADE',
                          'DATA_EXCLUSÃO',
                          'DR5']       

#Substituindo
for coluna in colunas_conv_data_base2:
  base_analise[coluna] = pd.to_datetime(base_analise[coluna], errors="coerce")

#Substituindo
for coluna in colunas_conv_data_base:
  base[coluna] = pd.to_datetime(base[coluna], errors="coerce")

Filtrando SIAPEs presentes em CACOPOSPRO e baseAnalise

In [79]:
#Merge entre a base e base2
servEncontradosCaco = base.merge(
    base_analise, on=['SIAPE'], 
    # how='outer',
    how='inner',
    suffixes=['', '_'], 
    indicator=True
    )

#Calculando a diferença entre a VIGENCIA INICIAL do CNR e a DATA DO REQUERIMENTO em dias
servEncontradosCaco['DiasDiffDR'] = (servEncontradosCaco['VIGENCIA_INICIAL'] - servEncontradosCaco['DATA_DO_REQUERIMENTO']).dt.days.abs()

#Ordena por SIAPE a diferença de dias
servEncontradosCacoDR = servEncontradosCaco.sort_values(by=['SIAPE', 'DiasDiffDR'])
servEncontradosCacoDR = servEncontradosCacoDR.groupby('SIAPE').first().reset_index()

#Renomeia as colunas para evitar conflitos no merge
servEncontradosCacoDR = servEncontradosCacoDR.rename(
    columns = {'Posicionamento':'NivelEmDR'}
    )
servEncontradosCaco = servEncontradosCaco.rename(
    columns = {'_merge':'MergeGeral'}
    )
servEncontradosCacoDR = servEncontradosCacoDR.rename(
    columns = {'_merge':'MergeDR'}
    )

servEncontradosCaco = servEncontradosCaco.rename(
    columns = {'Posicionamento':'NIVEL_POR_DATA'}
    )

#Merge entre servEncontradosCaco e servEncontradosCacoDR para identificar o CNR em DR nas proximas etapas do script
servEncontradosCaco = servEncontradosCaco.merge(
    servEncontradosCacoDR, on=['SIAPE'], 
    # how='outer',
    how='inner',
    suffixes=['', '_'], 
    indicator=True
    )


In [80]:
# Dicionário com as informações de mudança
substituicoesCodNivel = { '709-NS-D-101': 1, '707-NS-D-101': 1, 
                          '709-NS-D-102': 2, '707-NS-D-102': 2, 
                          '709-NS-D-201': 3, '707-NS-D-201': 3, 
                          '709-NS-D-202': 4, '707-NS-D-202': 4, 
                          '709-NS-D-301': 5, '707-NS-D-301': 5, 
                          '709-NS-D-302': 6, '707-NS-D-302': 6, 
                          '709-NS-D-303': 7, '707-NS-D-303': 7, 
                          '709-NS-D-304': 8, '707-NS-D-304': 8, 
                          '709-NS-D-401': 9, '707-NS-D-401': 9,
                          '709-NS-D-402':10, '707-NS-D-402': 10, 
                          '709-NS-D-403':11, '707-NS-D-403': 11, 
                          '709-NS-D-404':12, '707-NS-D-404': 12,
                          '704-NS-D-101': 1, '702-NA-D-101': 1, 
                          '704-NS-D-102': 2, '702-NA-D-102': 2, 
                          '704-NS-D-201': 3, '702-NA-D-201': 3,
                          '704-NS-D-202': 4, '702-NA-D-202': 4, 
                          '704-NS-D-301': 5, '702-NA-D-301': 5, 
                          '704-NS-D-302': 6, '702-NA-D-302': 6,
                          '704-NS-D-303': 7, '702-NA-D-303': 7, 
                          '704-NS-D-304': 8, '702-NA-D-304': 8,
                          '704-NS-D-401': 9, '702-NA-D-401': 9, 
                          '704-NS-D-402':10, '702-NA-D-402':10,
                          '704-NS-D-403':11, '702-NA-D-403':11, 
                          '704-NS-D-404':12, '702-NA-D-404':12}

#Substituindo
for valor_antigo, valor_novo in substituicoesCodNivel.items():
    servEncontradosCaco.loc[servEncontradosCaco['NivelEmDR'] == valor_antigo, 'COD_NIVELDR'] = valor_novo

# display(servEncontradosCaco)

In [81]:
servEncontradosCaco.loc[
    (servEncontradosCaco['VIGENCIA_INICIAL_'] > servEncontradosCaco['DATA_DO_REQUERIMENTO']), 'COD_NIVELDR'] = servEncontradosCaco['COD_NIVELDR'] - 1

In [82]:
# Dicionário com as informações de mudança
substituicoesCodNivel = { 1:101,  2:102, 
                          3:201,  4:202, 
                          5:301,  6:302, 
                          7:303,  8:304, 
                          9:401,  10:402, 
                          11:403, 12:404}

#Substituindo
for valor_antigo, valor_novo in substituicoesCodNivel.items():
    servEncontradosCaco.loc[servEncontradosCaco['COD_NIVELDR'] == valor_antigo, 'NIVEL_DR'] = valor_novo

# display(servEncontradosCaco)

Calculando a diferença entre DR5 e as datas de VIGENCIA INICIAL de cada Código de Carreira e Classe, por servidor. 

Em seguida, uma nova base é criada filtrada com a data e o Código de Carreira e Classe de cada servidor.

In [84]:
servEncontradosCaco['DiasDiff'] = (servEncontradosCaco['VIGENCIA_INICIAL'] - servEncontradosCaco['DR5']).dt.days.abs()

servEncontradosCaco = servEncontradosCaco.sort_values(by=['Nome', 'DiasDiff'])
servEncontradosCaco = servEncontradosCaco.groupby('Nome').first().reset_index()

Calculando (DTPi - DR5), para identificar o valor da diferença e filtrar a decisão.

In [85]:
servEncontradosCaco.loc[
    ((servEncontradosCaco['VIGENCIA_INICIAL'].dt.year - servEncontradosCaco['DR5'].dt.year <= 1) 
    & ((servEncontradosCaco['Cargo'] == '709001 PROFESSOR DO ENSINO BASICO EX-TERRITORIO')
    |(servEncontradosCaco['Cargo'] == '704001 PROFESSOR DO ENSINO BASICO EX-TERRITORIO'))), 'REGRA'] = 'EBF <= 1'

servEncontradosCaco.loc[
    ((servEncontradosCaco['VIGENCIA_INICIAL'].dt.year - servEncontradosCaco['DR5'].dt.year > 1) 
    & ((servEncontradosCaco['Cargo'] == '709001 PROFESSOR DO ENSINO BASICO EX-TERRITORIO')
    |(servEncontradosCaco['Cargo'] == '704001 PROFESSOR DO ENSINO BASICO EX-TERRITORIO'))), 'REGRA'] = 'EBF > 1' 

servEncontradosCaco.loc[
     ((servEncontradosCaco['VIGENCIA_INICIAL'].dt.year - servEncontradosCaco['DR5'].dt.year <= 1) 
     & ((servEncontradosCaco['Cargo'] == '707001 PROFESSOR ENS BASICO TECN TECNOL')
     |(servEncontradosCaco['Cargo'] == '702001 PROF DO ENSINO BASICO TEC,TECNOLOGICO'))), 'REGRA'] = 'EBTT <= 1'

servEncontradosCaco.loc[
     ((servEncontradosCaco['VIGENCIA_INICIAL'].dt.year - servEncontradosCaco['DR5'].dt.year > 1) 
     & ((servEncontradosCaco['Cargo'] == '707001 PROFESSOR ENS BASICO TECN TECNOL') 
     |(servEncontradosCaco['Cargo'] == '702001 PROF DO ENSINO BASICO TEC,TECNOLOGICO'))), 'REGRA'] = 'EBTT > 1'

servEncontradosCaco.loc[
     ((servEncontradosCaco['Cargo'] != '707001 PROFESSOR ENS BASICO TECN TECNOL') &
      (servEncontradosCaco['Cargo'] != '702001 PROF DO ENSINO BASICO TEC,TECNOLOGICO') &
      (servEncontradosCaco['Cargo'] != '709001 PROFESSOR DO ENSINO BASICO EX-TERRITORIO') &
      (servEncontradosCaco['Cargo'] != '704001 PROFESSOR DO ENSINO BASICO EX-TERRITORIO')), 'REGRA'] = 'MUDANÇA DE CARREIRA'

In [78]:
servEncontradosCacoDR.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1023 entries, 0 to 1022
Data columns (total 32 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   SIAPE                 1023 non-null   int64         
 1   Órgão                 1023 non-null   int64         
 2   CPF                   1023 non-null   int64         
 3   SiapeCad              1023 non-null   int64         
 4   Nome                  1023 non-null   object        
 5   Cargo                 1023 non-null   object        
 6   Cargo Inicial         1023 non-null   object        
 7   Cargo Final           1022 non-null   object        
 8   Posicionamento        1023 non-null   object        
 9   VIGENCIA_INICIAL      1021 non-null   datetime64[ns]
 10  VIGENCIA_FINAL        1020 non-null   datetime64[ns]
 11  Vigencia Status       1023 non-null   object        
 12  RESULTADO             1023 non-null   object        
 13  PROCESSO_SEI      

Tabelas de conversão de Código de Carreira e Classe (object) para um código de valor entre 1 - 12 (inteiro), isso facilitará o incremento.

In [86]:
# Dicionário com as informações de mudança
substituicoesCodNivel = { '709-NS-D-101': 1, '707-NS-D-101': 1, 
                          '709-NS-D-102': 2, '707-NS-D-102': 2, 
                          '709-NS-D-201': 3, '707-NS-D-201': 3, 
                          '709-NS-D-202': 4, '707-NS-D-202': 4, 
                          '709-NS-D-301': 5, '707-NS-D-301': 5, 
                          '709-NS-D-302': 6, '707-NS-D-302': 6, 
                          '709-NS-D-303': 7, '707-NS-D-303': 7, 
                          '709-NS-D-304': 8, '707-NS-D-304': 8, 
                          '709-NS-D-401': 9, '707-NS-D-401': 9,
                          '709-NS-D-402':10, '707-NS-D-402': 10, 
                          '709-NS-D-403':11, '707-NS-D-403': 11, 
                          '709-NS-D-404':12, '707-NS-D-404': 12,
                          '704-NS-D-101': 1, '702-NA-D-101': 1, 
                          '704-NS-D-102': 1, '702-NA-D-102': 1,
                          '704-NS-D-103': 2, '702-NA-D-103': 2,
                          '704-NS-D-104': 2, '702-NA-D-104': 2,      
                          '704-NS-D-201': 3, '702-NA-D-201': 3,
                          '704-NS-D-202': 3, '702-NA-D-202': 3, 
                          '704-NS-D-203': 4, '702-NA-D-203': 4,
                          '704-NS-D-204': 4, '702-NA-D-204': 4, 
                          '704-NS-D-301': 5, '702-NA-D-301': 5, 
                          '704-NS-D-302': 6, '702-NA-D-302': 6,
                          '704-NS-D-303': 7, '702-NA-D-303': 7, 
                          '704-NS-D-304': 8, '702-NA-D-304': 8,
                          '704-NS-D-401': 9, '702-NA-D-401': 9, 
                          '704-NS-D-402':10, '702-NA-D-402':10,
                          '704-NS-D-403':11, '702-NA-D-403':11, 
                          '704-NS-D-404':12, '702-NA-D-404':12}

#Substituindo
for valor_antigo, valor_novo in substituicoesCodNivel.items():
    servEncontradosCaco.loc[servEncontradosCaco['NIVEL_POR_DATA'] == valor_antigo, 'COD_NIVEL_DR5'] = valor_novo

# display(servEncontradosCaco)

In [87]:
servEncontradosCaco.loc[
    ((servEncontradosCaco['VIGENCIA_INICIAL'] > servEncontradosCaco['DR5'])&(servEncontradosCaco['COD_NIVEL_DR5'] > 1)), 'COD_NIVEL_DR5'] = servEncontradosCaco['COD_NIVEL_DR5'] - 1

Realização do incremento de acordo com o filtro da diferença entre DTPi - DR5.


In [88]:
servEncontradosCaco.loc[
    (servEncontradosCaco['REGRA'] == 'EBF <= 1'), 'INCREMENTO_CNR'] = servEncontradosCaco['COD_NIVEL_DR5'] + 3
servEncontradosCaco.loc[
    (servEncontradosCaco['REGRA'] == 'EBF > 1'), 'INCREMENTO_CNR'] = servEncontradosCaco['COD_NIVEL_DR5'] + 4


servEncontradosCaco.loc[
    (servEncontradosCaco['REGRA'] == 'EBTT <= 1'), 'INCREMENTO_CNR'] = servEncontradosCaco['COD_NIVEL_DR5'] + 2
servEncontradosCaco.loc[
    (servEncontradosCaco['REGRA'] == 'EBTT > 1'), 'INCREMENTO_CNR'] = servEncontradosCaco['COD_NIVEL_DR5'] + 3

Conversão do código de valores incrementados de acordo com a regra de diferença entre DTPi - DR5.

In [89]:
substituicoesCNR = { 1:101,  2:102, 
                     3:201,  4:202, 
                     5:301,  6:302, 
                     7:303,  8:304, 
                     9:401,  10:402, 
                     11:403}

# Substituindo valores menores que 12
for valor_antigo, valor_novo in substituicoesCNR.items():
    servEncontradosCaco.loc[servEncontradosCaco['INCREMENTO_CNR'] == valor_antigo, 'CNR'] = valor_novo
    
# Substituindo valores iguais ou maiores que 12
servEncontradosCaco.loc[servEncontradosCaco['INCREMENTO_CNR'] >= 12, 'CNR'] = 404

# display(servEncontradosCaco)

Descrição do resultado da análise.

In [90]:
servEncontradosCaco.loc[
    ((servEncontradosCaco['CNR'] < servEncontradosCaco['NIVEL_DR'])|
     (servEncontradosCaco['CNR'] > servEncontradosCaco['NIVEL_DR'])|
     (servEncontradosCaco['REGRA'] == 'MUDANÇA DE CARREIRA')), 'RESULTADO'] = 'A DECIDIR'
servEncontradosCaco.loc[
    (servEncontradosCaco['CNR'] == servEncontradosCaco['NIVEL_DR']), 'RESULTADO'] = 'INDEFERIDO'

servEncontradosCaco.loc[
    (servEncontradosCaco['CNR'] < servEncontradosCaco['NIVEL_DR']), 'CNRstatus'] = 'O CNR calculado é MENOR que o encontrado na Data do Requerimento'
servEncontradosCaco.loc[
    (servEncontradosCaco['CNR'] > servEncontradosCaco['NIVEL_DR']), 'CNRstatus'] = 'O CNR calculado é MAIOR que o encontrado na Data do Requerimento'
servEncontradosCaco.loc[
    (servEncontradosCaco['CNR'] == servEncontradosCaco['NIVEL_DR']), 'CNRstatus'] = 'O CNR calculado é IGUAL ao encontrado na Data do Requerimento'

In [91]:
colunas_conv_data = ['DATA_DO_REQUERIMENTO', 
                     'VIGENCIA_INICIAL', 
                     'VIGENCIA_FINAL',
                     'OBITO',
                     'INATIVIDADE',
                     'DATA_EXCLUSÃO']       

#Substituindo
for coluna in colunas_conv_data:
  servEncontradosCaco[coluna] = pd.to_datetime(servEncontradosCaco[coluna]).dt.strftime('%d/%m/%Y')

Geração do xlsx com os resultados.

In [92]:
# Lista de colunas para manter
colunas_manter = ['RESULTADO',  
                  'PROCESSO_SEI', 
                  'DATA_DO_REQUERIMENTO', 
                  'SIAPE', 
                  'INTERESSADO',
                  'NIVEL_POR_DATA',
                  'VIGENCIA_INICIAL',
                  'VIGENCIA_FINAL',
                  'SITUAÇÃO',
                  'LISTA',
                  'SERVIDOR',
                  'CPF',
                  'OBITO', 
                  'INATIVIDADE',
                  'CARGO',
                  'EXCLUSÃO',
                  'DATA_EXCLUSÃO',
                  'SITUAÇÃO_CADASTRO',
                  'ESTADO',
                  'DR5',
                  'NivelEmDR',
                  'CNRstatus']              											 				

# Exclusão das colunas que não estão na lista
servEncontradosCaco = servEncontradosCaco[colunas_manter]

# servEncontradosCaco.to_excel('ResultadoComplexo.xlsx')

In [93]:
resultado = servEncontradosCaco['RESULTADO'].value_counts().reset_index()
resultado.columns = ["Resumo Fluxo Complexo", "Quantidade"]

resultado_faltou      = servEncontradosCaco.query('RESULTADO == "ANÁLISE"').copy()
resultado_decidir     = servEncontradosCaco.query('RESULTADO == "A DECIDIR"').copy()
resultado_indeferido  = servEncontradosCaco.query('RESULTADO == "INDEFERIDO"').copy()

# display(resultado_faltou)

In [95]:
# Criar um arquivo Excel com ExcelWriter, usando a engine='xlsxwriter'
now = datetime.now()
Resultado_Complexo = f"05 Fluxo Complexo - Resultado {now.strftime('%Y-%m-%d %H:%M:%S')}.xlsx"
writer = pd.ExcelWriter(Resultado_Complexo, engine='xlsxwriter')

# Escreva os dataframes no arquivo Excel
servEncontradosCaco.to_excel  (writer, sheet_name ='COMPLEXO_RESULTADO', index=False)
resultado_faltou.to_excel(writer, sheet_name ='VERFICAR_NIVEL_5', index=False)
resultado_decidir.to_excel (writer, sheet_name ='DECIDIR_NIVEL_5', index=False)
resultado_indeferido.to_excel(writer, sheet_name ='INDEFERIDO_NIVEL_5', index=False)

resultado.to_excel(writer, sheet_name ='RESUMO_COMPLEXO', index=False)


cell_format   = writer.book.add_format({'align': 'center', 'valign': 'center'})
for row_num in range(len(resultado)):
    writer.sheets['RESUMO_COMPLEXO'].set_row(row_num, None, cell_format)

for col_num in range(len(resultado.columns)):
    col_width = resultado.iloc[:, col_num].astype(str).str.len().max()
    writer.sheets['RESUMO_COMPLEXO'].set_column(col_num, col_num, col_width, cell_format)

# Definir formato, filtros e cabeçalho para cada planilha
for sheet_name in ['COMPLEXO_RESULTADO', 'VERFICAR_NIVEL_5', 'DECIDIR_NIVEL_5','INDEFERIDO_NIVEL_5']:
    worksheet = writer.sheets[sheet_name]
  
    # Definir o formato de célula centralizado
    header_format = writer.book.add_format({'bold': True, 'bg_color': '#0070C0', 'color': 'white', 'align': 'center', 'valign': 'center'})
    worksheet.set_column(0, 0, options={'hidden': True})

    # Aplicar o formato de célula centralizado às colunas da planilha
    num_cols = len(servEncontradosCaco.columns)
    num_rows = len(servEncontradosCaco)

    # Ajustar a largura da coluna de modo a mostrar todo o texto
    for col_num in range(len(servEncontradosCaco.columns)):
        col_width = servEncontradosCaco.iloc[:, col_num].astype(str).str.len().max()
        worksheet.set_column(col_num, col_num, col_width, cell_format)

    for row_num in range(num_rows):
        worksheet.set_row(row_num, None, cell_format)

    # Adicionar filtros às colunas da planilha
    worksheet.autofilter(0, 0, num_rows - 1, num_cols - 1)

    # Congelar a primeira linha
    worksheet.freeze_panes(1, 0)
    
    # # Definir a cor azul para os nomes das colunas
    for col_num, value in enumerate(servEncontradosCaco.columns):
        worksheet.write(0, col_num, value, header_format)


# Salve o arquivo
writer.save()